## Define Your Model

In [1]:
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
model.eval().cpu()
print()

Downloading: "https://github.com/pytorch/vision/archive/v0.10.0.zip" to /root/.cache/torch/hub/v0.10.0.zip
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


  0%|          | 0.00/13.6M [00:00<?, ?B/s]

## Export to ONNX

In [4]:
x = torch.randn(1,3,224,224)
torch.onnx.export(model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "mobilenet.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                 )


## Install ONNX to NCNN

In [5]:
!wget -q https://github.com/Tencent/ncnn/releases/download/20220216/ncnn-20220216-ubuntu-1804.zip
!unzip -q ncnn-20220216-ubuntu-1804.zip

import os
os.environ['PATH'] += ":/content/ncnn-20220216-ubuntu-1804/bin"

## Convert ONNX to NCNN

In [6]:
!onnx2ncnn \
  mobilenet.onnx \
  mobilenet.param \
  mobilenet.bin

## Optimize NCNN model

In [7]:
!apt install vulkan-utils

# https://github.com/Tencent/ncnn/wiki/use-ncnnoptimize-to-optimize-model
!ncnnoptimize \
  mobilenet.param \
  mobilenet.bin \
  mobilenet-opt.param \
  mobilenet-opt.bin \
  65536

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libvulkan1 mesa-vulkan-drivers vulkan-tools
The following NEW packages will be installed:
  libvulkan1 mesa-vulkan-drivers vulkan-tools vulkan-utils
0 upgraded, 4 newly installed, 0 to remove and 49 not upgraded.
Need to get 3,315 kB of archives.
After this operation, 15.9 MB of additional disk space will be used.
Get:1 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic/main amd64 libvulkan1 amd64 1.1.126.0-2~gpu18.04.1 [101 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 mesa-vulkan-drivers amd64 20.0.8-0ubuntu1~18.04.1 [3,059 kB]
Get:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic/main amd64 vulkan-tools amd64 1.1.126.0+dfsg1-1~gpu18.04.1 [152 kB]
Get:4 ht

## Download result

In [8]:
!mkdir mobilenet
!cp mobilenet-opt.param mobilenet/mobilenet.param
!cp mobilenet-opt.bin mobilenet/mobilenet.bin

!zip -r mobilenet.zip mobilenet

from google.colab import files
files.download("mobilenet.zip")

  adding: mobilenet/ (stored 0%)
  adding: mobilenet/mobilenet.param (deflated 86%)
  adding: mobilenet/mobilenet.bin (deflated 8%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>